# Making adjustments to the model

Typically, we do not need to solve the models as they come from the authors
(someone else already did that!), but we want to perform various
perturbations in the model structure and conditions, and explore how the
model behaves in the changed conditions.

With COBREXA, there are 2 different approaches that one can take:
1. We can change the model structure and use the changed metabolic model.
   This is better for doing simple and small but systematic modifications,
   such as removing metabolites, adding reactions, etc.
2. We can intercept the pipeline that converts the metabolic model to
   constraints and then to the optimizer representation, and make small
   modifications along that way. This is better for various technical model
   adjustments, such as using combined objectives or adding reaction-coupling
   constraints.

Here we demonstrate the first, "modelling" approach. The main advantage of
that approach is that the modified model is still a FBC model, and you can
export, save and share it via the AbstractFBCModels interace. The main
disadvantage is that the "common" FBC model interface does not easily express
various complicated constructions (communities, reaction coupling, enzyme
constraints, etc.) -- see the [example about modifying the
constraints](02c-constraint-modifications.md) for a closer look on how to
modify even such complex constructions.

## Getting the base model

In [1]:
using COBREXA

download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

import JSONFBCModels

[ Info: using cached `e_coli_core.json'


For applying the modifications, we will use the canonical model as exported
from package `AbstractFBCModels`. There are other possibilities, but the
canonical one is easiest to use for common purposes.

In [2]:
import AbstractFBCModels.CanonicalModel as CM

We can now load the model:

In [3]:
model = convert(CM.Model, load_model("e_coli_core.json"))

AbstractFBCModels.CanonicalModel.Model(
  reactions = Dict{String, AbstractFBCModels.CanonicalModel.Reaction}("ACALD" =…
  metabolites = Dict{String, AbstractFBCModels.CanonicalModel.Metabolite}("glu_…
  genes = Dict{String, AbstractFBCModels.CanonicalModel.Gene}("b4301" => Abstra…
)


The canonical model is quite easy to work with, made basically of the most
accessible Julia structures possible. For example, you can observe a reaction
as such:

In [4]:
model.reactions["PFK"]

AbstractFBCModels.CanonicalModel.Reaction(
  name = "Phosphofructokinase",
  lower_bound = 0.0,
  upper_bound = 1000.0,
  stoichiometry = Dict("adp_c" => 1.0, "atp_c" => -1.0, "f6p_c" => -1.0, "fdp_c…
  objective_coefficient = 0.0,
  gene_association_dnf = [["b1723"], ["b3916"]],
  annotations = Dict("bigg.reaction" => ["PFK"], "metanetx.reaction" => ["MNXR1…
  notes = Dict("original_bigg_ids" => ["PFK"]),
)


In [5]:
model.reactions["CS"].stoichiometry

Dict{String, Float64} with 6 entries:
  "coa_c"   => 1.0
  "h2o_c"   => -1.0
  "accoa_c" => -1.0
  "cit_c"   => 1.0
  "oaa_c"   => -1.0
  "h_c"     => 1.0

## Running FBA on modified models

Since the canonical model is completely mutable, you can change it in any way you like and feed it directly into `flux_balance_analysis`. Let's first find a "original" solution, so that we have a base solution for comparing:

In [6]:
import GLPK

base_solution = flux_balance_analysis(model, GLPK.Optimizer)
base_solution.objective

0.8739215069684118

Now, for example, we can limit the intake of glucose by the model:

In [7]:
model.reactions["EX_glc__D_e"]

AbstractFBCModels.CanonicalModel.Reaction(
  name = "D-Glucose exchange",
  lower_bound = -10.0,
  upper_bound = 1000.0,
  stoichiometry = Dict("glc__D_e" => -1.0),
  objective_coefficient = 0.0,
  gene_association_dnf = nothing,
  annotations = Dict("sabiork" => ["7002", "601"], "metanetx.reaction" => ["MNX…
  notes = Dict("original_bigg_ids" => ["EX_glc_e"]),
)


Since the original intake limit is 10 units, let's try limiting that to 5:

In [8]:
model.reactions["EX_glc__D_e"].lower_bound = -5.0

-5.0

...and solve the modified model:

In [9]:
low_glucose_solution = flux_balance_analysis(model, GLPK.Optimizer)
low_glucose_solution.objective

0.4155977750928965

## Preventing reference-based sharing problems with `deepcopy`

People often want to try different perturbations with a single base model. It
would therefore look feasible to save retain the "unmodified" model in a
single variable, and make copies of that with the modifications applied.
Let's observe what happens:

In [10]:
base_model = convert(CM.Model, load_model("e_coli_core.json")) # load the base

modified_model = base_model # copy for modification

modified_model.reactions["EX_glc__D_e"].lower_bound = -123.0 # modify the glucose intake limit

-123.0

Surprisingly, the base model got modified too!

In [11]:
base_model.reactions["EX_glc__D_e"]

AbstractFBCModels.CanonicalModel.Reaction(
  name = "D-Glucose exchange",
  lower_bound = -123.0,
  upper_bound = 1000.0,
  stoichiometry = Dict("glc__D_e" => -1.0),
  objective_coefficient = 0.0,
  gene_association_dnf = nothing,
  annotations = Dict("sabiork" => ["7002", "601"], "metanetx.reaction" => ["MNX…
  notes = Dict("original_bigg_ids" => ["EX_glc_e"]),
)


This is because Julia uses reference-based sharing whenever anything mutable
is copied using the `=` operator. While this is extremely useful in many
scenarios for data processing efficiency and computational speed, it
unfortunately breaks this simple use-case.

To fix the situation, you should always ensure to make an actual copy of the
model data by either carefully copying the changed parts with `copy()`, or
simply by copying the whole model structure as is with `deepcopy()`. Let's
try again:

In [12]:
base_model = convert(CM.Model, load_model("e_coli_core.json"))
modified_model = deepcopy(base_model) # this forces an actual copy of the data
modified_model.reactions["EX_glc__D_e"].lower_bound = -123.0

-123.0

With `deepcopy`, the result works as intended:

In [13]:
(
    modified_model.reactions["EX_glc__D_e"].lower_bound,
    base_model.reactions["EX_glc__D_e"].lower_bound,
)

(-123.0, -10.0)

## Observing the differences

We already have a `base_solution` and `low_glucose_solution` from above. What
is the easiest way to see what has changed? We can quite easily compute
squared distance between all dictionary entries using Julia function for
merging dictionaries (called `mergewith`).

With that, we can extract the plain difference in fluxes:

In [14]:
flux_differences = mergewith(-, base_solution.fluxes, low_glucose_solution.fluxes)

ConstraintTrees.Tree{Float64} with 95 elements:
  :ACALD                    => -8.02663e-15
  :ACALDt                   => 0.0
  :ACKr                     => 1.9762e-14
  :ACONTa                   => 2.56379
  :ACONTb                   => 2.56379
  :ACt2r                    => 1.9762e-14
  :ADK1                     => 4.32987e-15
  :AKGDH                    => 2.06931
  :AKGt2r                   => 0.0
  :ALCD2x                   => -8.02663e-15
  :ATPM                     => 4.1922e-13
  :ATPS4r                   => 20.6429
  :BIOMASS_Ecoli_core_w_GAM => 0.458324
  :CO2t                     => -10.4958
  :CS                       => 2.56379
  :CYTBD                    => 19.9319
  :D_LACt2                  => -1.74927e-14
  :ENO                      => 7.13113
  :ETOHt2r                  => -8.02663e-15
  ⋮                         => ⋮

...and see what were the biggest directional differences:

In [15]:
sort(collect(flux_differences), by = last)

95-element Vector{Pair{Symbol, Union{Float64, ConstraintTrees.Tree{Float64}}}}:
        :H2Ot => -13.834417820645099
        :CO2t => -10.49580442817105
     :EX_o2_e => -9.965936361749776
         :PGK => -7.816778467951014
         :PGM => -7.1311261650652495
 :EX_glc__D_e => -4.999999999999955
    :EX_nh4_e => -2.4991476451708263
       :GLUDy => -2.381954266930253
      :SUCOAS => -2.069307205485625
     :EX_pi_e => -1.686035512450455
              ⋮
         :ENO => 7.1311261650652495
        :GAPD => 7.816778467951014
      :EX_h_e => 9.1939740614229
         :O2t => 9.965936361749776
    :EX_co2_e => 10.49580442817105
    :EX_h2o_e => 13.834417820645099
      :NADH16 => 17.86256551801395
       :CYTBD => 19.931872723499566
      :ATPS4r => 20.642858106791913

...or compute the squared distance, to see the "absolute" changes:

In [16]:
flux_changes =
    mergewith((x, y) -> (x - y)^2, base_solution.fluxes, low_glucose_solution.fluxes)

ConstraintTrees.Tree{Float64} with 95 elements:
  :ACALD                    => 6.44269e-29
  :ACALDt                   => 0.0
  :ACKr                     => 3.90535e-28
  :ACONTa                   => 6.57303
  :ACONTb                   => 6.57303
  :ACt2r                    => 3.90535e-28
  :ADK1                     => 1.87478e-29
  :AKGDH                    => 4.28203
  :AKGt2r                   => 0.0
  :ALCD2x                   => 6.44269e-29
  :ATPM                     => 1.75746e-25
  :ATPS4r                   => 426.128
  :BIOMASS_Ecoli_core_w_GAM => 0.210061
  :CO2t                     => 110.162
  :CS                       => 6.57303
  :CYTBD                    => 397.28
  :D_LACt2                  => 3.05995e-28
  :ENO                      => 50.853
  :ETOHt2r                  => 6.44269e-29
  ⋮                         => ⋮

...and again see what changed most:

In [17]:
sort(collect(flux_changes), by = last)

95-element Vector{Pair{Symbol, Union{Float64, ConstraintTrees.Tree{Float64}}}}:
      :ACALDt => 0.0
      :AKGt2r => 0.0
  :EX_acald_e => 0.0
    :EX_akg_e => 0.0
    :EX_fru_e => 0.0
    :EX_fum_e => 0.0
 :EX_gln__L_e => 0.0
 :EX_mal__L_e => 0.0
    :EX_pyr_e => 0.0
         :FBP => 0.0
              ⋮
     :EX_o2_e => 99.31988756644637
         :O2t => 99.31988756644637
        :CO2t => 110.16191059441503
    :EX_co2_e => 110.16191059441503
    :EX_h2o_e => 191.39111643618267
        :H2Ot => 191.39111643618267
      :NADH16 => 319.07124688534105
       :CYTBD => 397.27955026578604
      :ATPS4r => 426.1275908171446

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*